# Trabalho 3 - Image Classification
### Disciplina: Machine Learning II Professor: Carlos Padilha

### Alunos: Gustavo Costa / Vinícius Valim

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d jessicali9530/celeba-dataset

celeba-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import numpy as np
import pandas as pd

import os
import warnings
warnings.filterwarnings("ignore")
import random
import progressbar

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.applications.resnet50 import preprocess_input
from keras.layers import *

from PIL import Image
from keras.utils import Sequence

Using TensorFlow backend.


In [0]:
from zipfile import ZipFile
pasta_geral = '/content/celeba-dataset.zip'
zip_file = ZipFile(pasta_geral)

In [0]:
num_classes = 40
img_width = 224
img_height = 224
batch_size = 1024

In [0]:
arquivos = list(filter(lambda f: f.endswith("jpg"), zip_file.namelist()))

In [0]:
# Arquivo que para cada imagem possui 40 colunas identificando uma característica,
# como sombra, certos tipos de sobrancelhas etc. Os valores são -1 e 1
atributos = pd.read_csv(zip_file.open('list_attr_celeba.csv'))
atributos.set_index('image_id', inplace=True)

In [0]:
def load_images(list_images, idx, batch_size, net_w, net_h):
  
  train_data = []
  val_data =  []
  test_data =  []
  imgs = []

  partitions = pd.read_csv(zip_file.open('list_eval_partition.csv'))
  np.random.shuffle(list_images)

  lb = idx * batch_size
  rb = (idx + 1) * batch_size
  
  if rb > len(list_images):
      rb = len(list_images)
      lb = rb - batch_size
      
  batch_imgs = list_images[lb:rb]

  bar = progressbar.ProgressBar(maxval=batch_size, \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
  bar.start()
    
  for n in range(batch_size):
    with zip_file.open(batch_imgs[n]) as file:
      img = np.array(Image.open(file).resize((net_w, net_h)))

    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    aux = batch_imgs[n].split('/')[2]   # posição do nome do arquivo no Colab
    imgs.append(aux)

    filtro = partitions[partitions['image_id'] == aux]
    val = int(filtro['partition'])
    if val == 0:
        train_data.append(x)
    if val == 1:
        val_data.append(x)
    if val == 2:
        test_data.append(x)
    bar.update(n+1)
  
  train_data = np.vstack(train_data)
  val_data = np.vstack(val_data)
  test_data = np.vstack(test_data)

  bar.finish()

  return train_data, val_data, test_data, imgs


In [0]:
def load_batch(arquivos, idx, batch_size, img_width, img_height):
  train_data, val_data, test_data, imgs = load_images(list_images=arquivos,
                                                      idx=idx,
                                                      batch_size=batch_size,
                                                      net_w=img_width,
                                                      net_h=img_height)
  df_imagens = pd.DataFrame(imgs) 
  df_imagens = df_imagens.rename(columns = {0: "image_id"})
  df_imagens.set_index('image_id', inplace=True)

  img_split = pd.read_csv(zip_file.open('list_eval_partition.csv'))
  img_split.set_index('image_id', inplace=True)
  dataset = img_split.join(atributos, how="inner")

  dataset_amostra = dataset.join(df_imagens, how="inner")

  train_labels = dataset_amostra[dataset_amostra['partition'] == 0]
  train_labels = train_labels.drop("partition", axis=1)
  train_labels = np.array(train_labels)
  for i in range(len(train_labels)):
    for j in range(40):
      if train_labels[i][j] == -1:
          train_labels[i][j] = 0

  val_labels = dataset_amostra[dataset_amostra['partition'] == 1]
  val_labels = val_labels.drop("partition", axis=1)
  val_labels = np.array(val_labels)
  for i in range(len(val_labels)):
    for j in range(40):
      if val_labels[i][j] == -1:
          val_labels[i][j] = 0

  test_labels = dataset_amostra[dataset_amostra['partition'] == 2]
  test_labels = test_labels.drop("partition", axis=1)
  test_labels = np.array(test_labels)
  for i in range(len(test_labels)):
    for j in range(40):
      if test_labels[i][j] == -1:
          test_labels[i][j] = 0

  return train_data, val_data, test_data, train_labels, val_labels, test_labels, imgs

In [10]:
train_data, val_data, test_data, train_labels, val_labels, test_labels, imgs = load_batch(arquivos=arquivos, 
                                                                                          idx=0, 
                                                                                          batch_size=batch_size, 
                                                                                          img_width=img_width, 
                                                                                          img_height=img_height)

train_data.shape, val_data.shape, test_data.shape, train_labels.shape, val_labels.shape, test_labels.shape

[========================================================================] 100%


((827, 224, 224, 3),
 (104, 224, 224, 3),
 (93, 224, 224, 3),
 (827, 40),
 (104, 40),
 (93, 40))

# Aplicação do Modelo

In [0]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras import models
from keras import layers
from keras import optimizers

In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [13]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(img_width, img_height, 3))
                  
model = Sequential()
model.add(conv_base)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))





58892288/58889256 [==============================] - 1s 0us/step







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
conv_base.trainable = False

In [15]:

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
total_batches = int(np.ceil(float(len(arquivos)) / batch_size))
for i in range(total_batches):
    train_data, val_data, test_data, train_labels, val_labels, test_labels, imgs = load_batch(arquivos=arquivos, 
                                                                                              idx=i, 
                                                                                              batch_size=batch_size, 
                                                                                              img_width=img_width, 
                                                                                              img_height=img_height)

    print(train_data.shape, val_data.shape, test_data.shape, train_labels.shape, val_labels.shape, test_labels.shape)
    print('Treinando lote ', i+1, ' de ', total_batches)

    train_datagen.fit(train_data)

    train_generator = train_datagen.flow(train_data, train_labels, batch_size=32)
    val_generator = val_datagen.flow(val_data, val_labels, batch_size=32)
    test_generator = test_datagen.flow(test_data, test_labels, batch_size=32)

    result = model.fit_generator(
            train_generator,
            steps_per_epoch=20,
            epochs=10,
            validation_data=val_generator,
            validation_steps=10,
            verbose=1)

[========================================================================] 100%


(825, 224, 224, 3) (108, 224, 224, 3) (91, 224, 224, 3) (825, 40) (108, 40) (91, 40)
Treinando lote  1  de  198


Epoch 1/10
20/20 [==============================] - 18s 903ms/step - loss: 0.8707 - acc: 0.4919 - val_loss: 0.7060 - val_acc: 0.5601
Epoch 2/10
20/20 [==============================] - 10s 479ms/step - loss: 0.7496 - acc: 0.5696 - val_loss: 0.6209 - val_acc: 0.6554
Epoch 3/10
20/20 [==============================] - 9s 426ms/step - loss: 0.6672 - acc: 0.6348 - val_loss: 0.5600 - val_acc: 0.7263
Epoch 4/10
20/20 [==============================] - 8s 401ms/step - loss: 0.6208 - acc: 0.6762 - val_loss: 0.5149 - val_acc: 0.7649
Epoch 5/10
20/20 [==============================] - 8s 392ms/step - loss: 0.5733 - acc: 0.7190 - val_loss: 0.4909 - val_acc: 0.7846
Epoch 6/10
20/20 [==============================] - 9s 450ms/step - loss: 0.5426 - acc: 0.7453 - val_loss: 0.4712 - val_acc: 0.7938
Epoch 7/10
20/20 [==============================] - 9s 435ms/step - loss: 0.5266 - acc: 0.75

[========================================================================] 100%


(833, 224, 224, 3) (88, 224, 224, 3) (103, 224, 224, 3) (833, 40) (88, 40) (103, 40)
Treinando lote  2  de  198
Epoch 1/10
20/20 [==============================] - 13s 628ms/step - loss: 0.4974 - acc: 0.7811 - val_loss: 0.4737 - val_acc: 0.7819
Epoch 2/10
20/20 [==============================] - 9s 439ms/step - loss: 0.4948 - acc: 0.7825 - val_loss: 0.4740 - val_acc: 0.7845
Epoch 3/10
20/20 [==============================] - 9s 436ms/step - loss: 0.4991 - acc: 0.7825 - val_loss: 0.4753 - val_acc: 0.7846
Epoch 4/10
20/20 [==============================] - 8s 397ms/step - loss: 0.4955 - acc: 0.7846 - val_loss: 0.4776 - val_acc: 0.7843
Epoch 5/10
20/20 [==============================] - 10s 478ms/step - loss: 0.4892 - acc: 0.7862 - val_loss: 0.4737 - val_acc: 0.7867
Epoch 6/10
20/20 [==============================] - 9s 433ms/step - loss: 0.5017 - acc: 0.7790 - val_loss: 0.4778 - val_acc: 0.7844
Epoch 7/10
20/20 [==============================] - 9s 434ms/step - loss: 0.4918 - acc: 0.7866

[========================================================================] 100%


...

(834, 224, 224, 3) (104, 224, 224, 3) (86, 224, 224, 3) (834, 40) (104, 40) (86, 40)
Treinando lote  196  de  198
Epoch 1/10
20/20 [==============================] - 9s 474ms/step - loss: 0.4493 - acc: 0.8006 - val_loss: 0.4389 - val_acc: 0.8184
Epoch 2/10
20/20 [==============================] - 9s 429ms/step - loss: 0.4492 - acc: 0.8046 - val_loss: 0.4416 - val_acc: 0.8143
Epoch 3/10
20/20 [==============================] - 8s 415ms/step - loss: 0.4392 - acc: 0.8059 - val_loss: 0.4393 - val_acc: 0.8176
Epoch 4/10
20/20 [==============================] - 8s 390ms/step - loss: 0.4459 - acc: 0.8068 - val_loss: 0.4398 - val_acc: 0.8185
Epoch 5/10
20/20 [==============================] - 9s 462ms/step - loss: 0.4458 - acc: 0.8044 - val_loss: 0.4317 - val_acc: 0.8203
Epoch 6/10
20/20 [==============================] - 8s 422ms/step - loss: 0.4504 - acc: 0.8072 - val_loss: 0.4476 - val_acc: 0.8167
Epoch 7/10
20/20 [==============================] - 8s 411ms/step - loss: 0.4364 - acc: 0.8102

[========================================================================] 100%


(809, 224, 224, 3) (115, 224, 224, 3) (100, 224, 224, 3) (809, 40) (115, 40) (100, 40)
Treinando lote  197  de  198
Epoch 1/10
20/20 [==============================] - 10s 488ms/step - loss: 0.4543 - acc: 0.8030 - val_loss: 0.4320 - val_acc: 0.8081
Epoch 2/10
20/20 [==============================] - 9s 467ms/step - loss: 0.4469 - acc: 0.8027 - val_loss: 0.4324 - val_acc: 0.8112
Epoch 3/10
20/20 [==============================] - 8s 420ms/step - loss: 0.4504 - acc: 0.8014 - val_loss: 0.4376 - val_acc: 0.8082
Epoch 4/10
20/20 [==============================] - 8s 413ms/step - loss: 0.4528 - acc: 0.8003 - val_loss: 0.4272 - val_acc: 0.8122
Epoch 5/10
20/20 [==============================] - 8s 409ms/step - loss: 0.4539 - acc: 0.8010 - val_loss: 0.4364 - val_acc: 0.8094
Epoch 6/10
20/20 [==============================] - 9s 463ms/step - loss: 0.4541 - acc: 0.8042 - val_loss: 0.4284 - val_acc: 0.8093
Epoch 7/10
20/20 [==============================] - 9s 448ms/step - loss: 0.4452 - acc: 0.8

[========================================================================] 100%


(829, 224, 224, 3) (101, 224, 224, 3) (94, 224, 224, 3) (829, 40) (101, 40) (94, 40)
Treinando lote  198  de  198
Epoch 1/10
20/20 [==============================] - 10s 513ms/step - loss: 0.4512 - acc: 0.8026 - val_loss: 0.4469 - val_acc: 0.8035
Epoch 2/10
20/20 [==============================] - 9s 453ms/step - loss: 0.4628 - acc: 0.7991 - val_loss: 0.4526 - val_acc: 0.8006
Epoch 3/10
20/20 [==============================] - 9s 433ms/step - loss: 0.4450 - acc: 0.8069 - val_loss: 0.4482 - val_acc: 0.8025
Epoch 4/10
20/20 [==============================] - 8s 400ms/step - loss: 0.4615 - acc: 0.7994 - val_loss: 0.4522 - val_acc: 0.7988
Epoch 5/10
20/20 [==============================] - 8s 394ms/step - loss: 0.4550 - acc: 0.7989 - val_loss: 0.4482 - val_acc: 0.8032
Epoch 6/10
20/20 [==============================] - 9s 456ms/step - loss: 0.4518 - acc: 0.8010 - val_loss: 0.4526 - val_acc: 0.7990
Epoch 7/10
20/20 [==============================] - 9s 429ms/step - loss: 0.4528 - acc: 0.805

In [17]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=30)
score = model.evaluate_generator(test_generator, steps=30)
print("Acurácia:", score[1]*100)

Acurácia: 79.62765934619497


In [0]:
model.save('/content/keras.model')